In [1]:
import fastf1
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path

In [2]:
def get_data_path(filename):
    """Get full path for data file"""
    data_dir = Path('./data')
    data_dir.mkdir(parents=True, exist_ok=True)
    return data_dir / filename

In [3]:
def collect_race_data(year):
    """Collect comprehensive race data including pit stops"""
    fastf1.Cache.enable_cache('./data/fastf1_cache')
    features = []
    labels = []
   
    print(f"Processing year {year}")
   
    for round_no in range(1, 30):  # Loop through potential rounds
        try:
            # Load race session
            race_session = fastf1.get_session(year, round_no, 'R')
            race_session.load(telemetry=False)
           
            # Load qualifying session
            quali_session = fastf1.get_session(year, round_no, 'Q')
            quali_session.load(telemetry=False)
           
            print(f"Processing {race_session.event['EventName']}")
           
            # Get all laps data
            laps = race_session.laps
           
            # Get stint data
            stints = laps[["Driver", "Stint", "Compound", "LapNumber"]]
            stints = stints.groupby(["Driver", "Stint", "Compound"])
            stints = stints.count().reset_index()
           
            for driver in race_session.drivers:
                try:
                    # Get driver session information
                    driver_results = race_session.get_driver(driver)
                   
                    if driver_results is None:
                        continue
                       
                    # Skip if driver was disqualified
                    if driver_results['Position'] in ['D', 'DSQ']:
                        print(f"Skipping disqualified driver {driver}")
                        continue
                       
                    driver_abbr = driver_results["Abbreviation"]
                    position = driver_results['Position']  # Move this up before we use it
                    driver_stints = stints[stints["Driver"] == driver_abbr]
                   
                    # Get ordered list of compounds used
                    compounds = driver_stints["Compound"].tolist()
                   
                    # Get pit stop information
                    driver_laps = race_session.laps.pick_drivers(driver)
                    pit_stops = []
                    valid_entry = True  # Flag to check if entry should be added to dataset
                   
                    # Look for pit stops in consecutive laps
                    for i in range(len(driver_laps) - 1):
                        current_lap = driver_laps.iloc[i]
                        next_lap = driver_laps.iloc[i + 1]
                       
                        if pd.notnull(current_lap['PitInTime']) and pd.notnull(next_lap['PitOutTime']):
                            try:
                                duration = (next_lap['PitOutTime'] - current_lap['PitInTime']).total_seconds()
                                if 15 <= duration <= 45:  # Typical total pit lane time
                                    pit_stops.append(int(current_lap['LapNumber']))  # Convert to regular integer
                                else:
                                    valid_entry = False  # Invalid pit stop duration
                                    print(f"Excluding entry - Invalid pit duration {duration:.2f}s for {driver_abbr}")
                                    break
                            except Exception as e:
                                valid_entry = False
                                print(f"Error calculating pit duration for {driver_abbr}: {e}")
                                break
                   
                    if not valid_entry:
                        continue  # Skip this driver's entry
                       
                    # Skip entries where driver finished but had no pit stops
                    try:
                        pos_int = int(position)  # This will succeed only for classified finishers
                        if len(pit_stops) == 0:
                            print(f"Excluding entry - Classified finish but no pit stops for {driver_abbr}")
                            continue
                    except (ValueError, TypeError):
                        # For non-classified results (DNF, DSQ, etc), we can keep 0 pit stop entries
                        pass
                   
                    # Get qualifying data
                    quali_results = quali_session.get_driver(driver)
                   
                    # Handle grid position and classification result
                    grid_position = driver_results['GridPosition']
                    points = driver_results['Points']
                   
                    # Only include classified results
                    try:
                        pos_int = int(position)
                        classification_result = pos_int - grid_position
                    except (ValueError, TypeError):
                        # Skip non-classified results
                        continue
                   
                    # Get number of participants
                    num_participants = len(race_session.results)
                   
                    feature = {
                        'Season': year,
                        'Round': round_no,
                        'Circuit': race_session.event['Location'] if 'Location' in race_session.event else None,
                        'Laps': race_session.laps['LapNumber'].max(),
                        'NumParticipants': num_participants,
                        'Driver': driver_abbr,
                        'DriverNumber': driver,
                        'Team': driver_results['TeamName'],
                        'GridPosition': grid_position,
                        'QualifyingPosition': quali_results['Position'] if quali_results is not None else None,
                        'Q1Time': quali_results['Q1'].total_seconds() if quali_results is not None and pd.notnull(quali_results['Q1']) else None,
                        'AirTemp': race_session.weather_data['AirTemp'].iloc[0] if isinstance(race_session.weather_data['AirTemp'], pd.Series) else np.nan,
                        'Humidity': race_session.weather_data['Humidity'].iloc[0] if isinstance(race_session.weather_data['Humidity'], pd.Series) else np.nan,
                        'Pressure': race_session.weather_data['Pressure'].iloc[0] if isinstance(race_session.weather_data['Pressure'], pd.Series) else np.nan,
                        'TrackTemp': race_session.weather_data['TrackTemp'].iloc[0] if isinstance(race_session.weather_data['TrackTemp'], pd.Series) else np.nan,
                        'WindDirection': race_session.weather_data['WindDirection'].iloc[0] if isinstance(race_session.weather_data['WindDirection'], pd.Series) else np.nan,
                        'WindSpeed': race_session.weather_data['WindSpeed'].iloc[0] if isinstance(race_session.weather_data['WindSpeed'], pd.Series) else np.nan,
                    }
                   
                    label = {
                        'NumPitStops': len(pit_stops),
                        'PitStopLaps': pit_stops,  # Now storing as regular list of integers
                        'TyreCompounds': compounds,
                        'ClassificationResult': classification_result,
                        'Points': points,
                    }
                   
                    features.append(feature)
                    labels.append(label)
                   
                    print(f"Processed data for {driver_abbr} - Found {len(pit_stops)} valid pit stops")
                    if pit_stops:
                        print(f"  Pit stop laps: {pit_stops}")
                   
                except Exception as e:
                    print(f"Error processing driver {driver} at {race_session.event['EventName']}: {str(e)}")
                    continue
                   
        except Exception as e:
            print(f"Could not process round {round_no} of {year}: {e}")
            if round_no > 20:  # Most seasons have fewer than 20 races
                break
            else:
                continue
   
    df_features = pd.DataFrame(features)
    df_labels = pd.DataFrame(labels)
   
    print(f"\nCollected data for {len(df_features)} driver-race combinations")
    return df_features, df_labels

def save_race_data(df_features, df_labels, year):
    """Save the collected data to a single CSV file in the data directory"""
    # Reset index to ensure proper alignment
    df_features = df_features.reset_index(drop=True)
    df_labels = df_labels.reset_index(drop=True)
    
    # Combine features and labels
    combined_df = pd.concat([df_features, df_labels], axis=1)
    
    # Save to single CSV
    filepath = get_data_path(f'f1_data_{year}.csv')
    combined_df.to_csv(filepath, index=False)
    print(f"Data saved to {filepath}")
    
    return combined_df

def analyze_data(combined_df, year):
    """Analyze the collected data and print summary statistics"""
    print("\nDataset Summary:")
    print(f"Total races: {combined_df['Round'].nunique()}")
    print(f"Total drivers: {combined_df['Driver'].nunique()}")
    print(f"Years covered: {combined_df['Season'].min()} to {combined_df['Season'].max()}")
    
    print("\nPit Stop Statistics:")
    print(f"Average pit stops per race: {combined_df['NumPitStops'].mean():.2f}")
    print(f"Maximum pit stops in a race: {combined_df['NumPitStops'].max()}")
    
    # Save analysis results
    analysis_path = get_data_path(f'analysis_summary_{year}.txt')
    with open(analysis_path, 'w') as f:
        f.write("Dataset Analysis Summary\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Total races: {combined_df['Round'].nunique()}\n")
        f.write(f"Total drivers: {combined_df['Driver'].nunique()}\n")
        f.write(f"Years covered: {combined_df['Season'].min()} to {combined_df['Season'].max()}\n\n")
        f.write("Missing Data Analysis:\n")
        f.write(combined_df.isnull().sum()[combined_df.isnull().sum() > 0].to_string())
    
    return combined_df.describe()

In [4]:
year = 2024
print("Starting filtered F1 data collection...")
df_features, df_labels = collect_race_data(year)
combined_df = save_race_data(df_features, df_labels, year)
stats = analyze_data(combined_df, year)
print("\nData Statistics:")
print(stats)
print("\nData collection and analysis complete!")

Starting filtered F1 data collection...
Processing year 2024


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached da

Processing Bahrain Grand Prix
Processed data for VER - Found 2 valid pit stops
  Pit stop laps: [17, 37]
Processed data for PER - Found 2 valid pit stops
  Pit stop laps: [12, 36]
Processed data for SAI - Found 2 valid pit stops
  Pit stop laps: [14, 35]
Processed data for LEC - Found 2 valid pit stops
  Pit stop laps: [11, 34]
Processed data for RUS - Found 2 valid pit stops
  Pit stop laps: [11, 31]
Processed data for NOR - Found 2 valid pit stops
  Pit stop laps: [13, 33]
Processed data for HAM - Found 2 valid pit stops
  Pit stop laps: [12, 33]
Processed data for PIA - Found 2 valid pit stops
  Pit stop laps: [12, 34]


core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Processed data for ALO - Found 2 valid pit stops
  Pit stop laps: [15, 41]
Processed data for STR - Found 2 valid pit stops
  Pit stop laps: [9, 27]
Processed data for ZHO - Found 2 valid pit stops
  Pit stop laps: [9, 28]
Processed data for MAG - Found 2 valid pit stops
  Pit stop laps: [11, 32]
Processed data for RIC - Found 2 valid pit stops
  Pit stop laps: [13, 35]
Processed data for TSU - Found 2 valid pit stops
  Pit stop laps: [14, 34]
Processed data for ALB - Found 2 valid pit stops
  Pit stop laps: [15, 36]
Processed data for HUL - Found 3 valid pit stops
  Pit stop laps: [1, 20, 41]
Processed data for OCO - Found 2 valid pit stops
  Pit stop laps: [10, 30]
Processed data for GAS - Found 3 valid pit stops
  Pit stop laps: [12, 31, 43]
Excluding entry - Invalid pit duration 74.95s for BOT
Processed data for SAR - Found 3 valid pit stops
  Pit stop laps: [10, 28, 40]


...

Processing Qatar Grand Prix
Processed data for VER - Found 3 valid pit stops
  Pit stop laps: [35, 36, 37]
Processed data for LEC - Found 3 valid pit stops
  Pit stop laps: [35, 36, 37]
Processed data for PIA - Found 3 valid pit stops
  Pit stop laps: [34, 36, 37]
Processed data for RUS - Found 4 valid pit stops
  Pit stop laps: [23, 35, 36, 37]
Processed data for GAS - Found 3 valid pit stops
  Pit stop laps: [35, 36, 37]
Processed data for SAI - Found 3 valid pit stops
  Pit stop laps: [34, 36, 37]
Processed data for ALO - Found 3 valid pit stops
  Pit stop laps: [35, 36, 37]
Processed data for ZHO - Found 3 valid pit stops
  Pit stop laps: [35, 36, 37]
Processed data for MAG - Found 3 valid pit stops
  Pit stop laps: [27, 36, 37]


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Processed data for NOR - Found 4 valid pit stops
  Pit stop laps: [35, 36, 37, 45]
Processed data for BOT - Found 3 valid pit stops
  Pit stop laps: [24, 35, 36]
Processed data for HAM - Found 4 valid pit stops
  Pit stop laps: [34, 35, 36, 47]
Processed data for TSU - Found 4 valid pit stops
  Pit stop laps: [35, 36, 37, 40]
Excluding entry - Invalid pit duration 45.05s for LAW
Processed data for ALB - Found 4 valid pit stops
  Pit stop laps: [35, 36, 37, 55]
Processed data for HUL - Found 4 valid pit stops
  Pit stop laps: [1, 31, 36, 37]
Processed data for PER - Found 3 valid pit stops
  Pit stop laps: [35, 36, 37]
Processed data for STR - Found 2 valid pit stops
  Pit stop laps: [1, 7]
Excluding entry - Classified finish but no pit stops for COL
Excluding entry - Classified finish but no pit stops for OCO


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached dat

Processing Abu Dhabi Grand Prix
Processed data for NOR - Found 1 valid pit stops
  Pit stop laps: [26]
Processed data for SAI - Found 1 valid pit stops
  Pit stop laps: [25]
Processed data for LEC - Found 1 valid pit stops
  Pit stop laps: [20]
Processed data for HAM - Found 1 valid pit stops
  Pit stop laps: [34]
Processed data for RUS - Found 1 valid pit stops
  Pit stop laps: [26]
Processed data for VER - Found 1 valid pit stops
  Pit stop laps: [29]
Processed data for GAS - Found 1 valid pit stops
  Pit stop laps: [14]
Processed data for HUL - Found 1 valid pit stops
  Pit stop laps: [13]
Processed data for ALO - Found 2 valid pit stops
  Pit stop laps: [13, 37]
Processed data for PIA - Found 2 valid pit stops
  Pit stop laps: [4, 32]
Processed data for ALB - Found 1 valid pit stops
  Pit stop laps: [12]
Processed data for TSU - Found 1 valid pit stops
  Pit stop laps: [22]
Processed data for ZHO - Found 2 valid pit stops
  Pit stop laps: [12, 39]
Processed data for STR - Found 2 v